<a href="https://colab.research.google.com/github/SimoneDutto/MachineLearning/blob/master/HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


**Install requirements**

In [0]:
!pip3 install 'torch==1.3.1'
!pip3 install 'torchvision==0.4.2'
!pip3 install 'Pillow-SIMD'
!pip3 install 'tqdm'

     |████████████████████████████████| 624kB 3.5MB/s 
  Created wheel for Pillow-SIMD: filename=Pillow_SIMD-6.0.0.post0-cp36-cp36m-linux_x86_64.whl size=1062854 sha256=ce2da5931426798cc253d3314635b2bbf9d68ea166d84faec1352f3b43cf63e1
  Stored in directory: /root/.cache/pip/wheels/06/60/65/cc9afa345ccbf10a34cc208266b992941a8608010b592f43d1
Successfully built Pillow-SIMD


**Import libraries**

In [0]:
import os
import logging
import os.path
import sys

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Subset, DataLoader
from torch.backends import cudnn

import torchvision
from torchvision import transforms
from torchvision.models import alexnet, resnet50

from PIL import Image
from tqdm import tqdm
from torchvision.datasets import VisionDataset

from PIL import Image


** Set Arguments**

*   No pretrain LR = 1e-2 STEPSIZE =26 55
*   Pretrain LR = 1e-2 STEPSIZE = 10 EPOCHS = 20 86
*   Freeze AlexNet 83
*   Freeze fully connect 0.5911485774499473 LR = 1e-2 STEPSIZE = 20 EPOCHS = 30
*   RANDOM Horizontal flip LR = 1e-2 STEPSIZE = 10 EPOCHS = 20 86
*   RANDOM Color Brightness LR = 1e-2 STEPSIZE = 10 EPOCHS = 20 86.6
*   RANDOM Ten Crop LR = 1e-3 BATCH 128 NUM_EPOCHS = 30 STEP_SIZE =20 GAMMA = 0.1 84.5
* ResNet


In [0]:
DEVICE = 'cuda' # 'cuda' or 'cpu'

NUM_CLASSES = 102 # 101 + 1: There is am extra Background class that should be removed 

BATCH_SIZE = 256    # Higher batch sizes allows for larger learning rates. An empirical heuristic suggests that, when changing
                     # the batch size, learning rate should change by the same factor to have comparable results

LR = 1e-2            # The initial Learning Rate 
MOMENTUM = 0.9       # Hyperparameter for SGD, keep this at 0.9 when using SGD
WEIGHT_DECAY = 5e-5  # Regularization, you can keep this at the default

NUM_EPOCHS = 20      # Total number of training epochs (iterations over dataset)
STEP_SIZE = 10    # How many epochs before decreasing learning rate (if using a step-down policy)
GAMMA = 0.1          # Multiplicative factor for learning rate step-down

LOG_FREQUENCY = 10
DATA_DIR = 'Homework2-Caltech101/101_ObjectCategories/'
FILE_DIR = 'Homework2-Caltech101/'

**Define Data Preprocessing**

In [0]:
# Define transforms for training phase
train_transform = transforms.Compose([transforms.Resize(256),      # Resizes short size of the PIL image to 256
                                      transforms.CenterCrop(224),  # Crops a central square patch of the image
                                                                   # 224 because torchvision's AlexNet needs a 224x224 input!
                                                                   # Remember this when applying different transformations, otherwise you get an error
                                      #transforms.transforms.RandomHorizontalFlip(),
                                      #transforms.ColorJitter(0.5, 0.7),
                                      #transforms.RandomRotation(20),
                                      transforms.ToTensor(), # Turn PIL Image to torch.Tensor
                                      #transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                                      transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225) ) # Normalizes tensor with mean and standard deviation
])
# Define transforms for the evaluation phase
eval_transform = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      #transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                                      transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225) )
                                      #transforms.TenCrop(224), # this is a list of PIL Images
                                      #transforms.Lambda(lambda crops: torch.stack([transforms.ToTensor()(crop) for crop in crops])),
                                      #transforms.Lambda(lambda crops: torch.stack([transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])(crop) for crop in crops])),                                    
])
test_transform = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      #transforms.ToTensor(),
                                      #transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                                      #transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225) )
                                      transforms.TenCrop(224), # this is a list of PIL Images
                                      transforms.Lambda(lambda crops: torch.stack([transforms.ToTensor()(crop) for crop in crops])),
                                      transforms.Lambda(lambda crops: torch.stack([transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])(crop) for crop in crops])),                                    
])


**Class to define the split set**

In [0]:
def pil_loader(path):
    # open path as file to avoid ResourceWarning (https://github.com/python-pillow/Pillow/issues/835)
    with open(DATA_DIR+path, 'rb') as f:
        img = Image.open(f)
        return img.convert('RGB')

def divideClasses():
    class_label = dict(tuple())
    num_label = 0
    train_paths =list()
    val_paths = list()
    test_paths = list()    
    f = open(FILE_DIR+"train.txt", "r")
    for x in f:
      if not x.startswith("BACKGROUND"):
          label = x.split("/")[0]
          if label not in class_label:
              des_label = num_label
              class_label[label] = des_label,1
              train_paths.append(x.rstrip()+"-"+str(des_label))
              num_label+=1
          else:
              des_label, flag = class_label[label]
              if flag < 2:
                train_paths.append(x.rstrip()+"-"+str(des_label))
                flag += 1
                class_label[label] = des_label,flag
              else:
                val_paths.append(x.rstrip()+"-"+str(des_label))
                class_label[label] = des_label,0
    f.close()
    f = open(FILE_DIR+"test.txt", "r")
    i = 0
    for x in f:
      if not x.startswith("BACKGROUND"):
          label = x.split("/")[0]
          des_label, flag = class_label[label]
          test_paths.append(x.rstrip()+"-"+str(des_label))
          i += 1
    f.close()
    return train_paths, val_paths, test_paths
class Caltech(VisionDataset):
    split = str() 
    paths = list()
    def __init__(self, root, split, transform=None, target_transform=None, paths=None):

        super(Caltech, self).__init__(root, transform=transform, target_transform=target_transform)

        self.split = split # This defines the split you are going to use
                           # (split files are called 'train.txt' and 'test.txt')
        
        self.paths = paths
        '''
        - Here you should implement the logic for reading the splits files and accessing elements
        - If the RAM size allows it, it is faster to store all data in memory
        - PyTorch Dataset classes use indexes to read elements
        - You should provide a way for the __getitem__ method to access the image-label pair
          through the index
        - Labels should start from 0, so for Caltech you will have lables 0...100 (excluding the background class) 
        '''

    def __getitem__(self, index):
        '''
        __getitem__ should access an element through its index
        Args:
            index (int): Index
        Returns:
            tuple: (sample, target) where target is class_index of the target class.
        '''
        
        im_lb = self.paths[index].split("-")
        
        image, label = pil_loader(im_lb[0]), int(im_lb[1])

        # Applies preprocessing when accessing the image
        if self.transform is not None:
            image = self.transform(image)

        return image, label

    def __len__(self):
        '''
        The __len__ method returns the length of the dataset
        It is mandatory, as this is used by several other components
        '''
        length = len(self.paths)
        return length

**Prepare Dataset**

In [0]:
# Clone github repository with data
if not os.path.isdir('./Homework2-Caltech101'):
  !git clone https://github.com/MachineLearning2020/Homework2-Caltech101.git


#Prepare Pytorch train/test Datasets
# train_dataset = torchvision.datasets.ImageFolder(DATA_DIR, transform=train_transform)
# test_dataset = torchvision.datasets.ImageFolder(DATA_DIR, transform=eval_transform)

# train_indexes = [idx for idx in range(len(train_dataset)) if idx % 5]
# test_indexes = [idx for idx in range(len(test_dataset)) if not idx % 5]

# train_dataset = Subset(train_dataset, train_indexes)
# test_dataset = Subset(test_dataset, test_indexes)

# print(len(train_dataset)+len(test_dataset))

train_paths, val_paths, test_paths  = divideClasses()
train_dataset = Caltech("2",split="train",transform=train_transform, paths=train_paths)
val_dataset = Caltech("3",split="val",transform=eval_transform, paths=val_paths)
test_dataset = Caltech("4",split="test", transform=test_transform, paths = test_paths)
print(train_dataset.transform)
# Check dataset sizes
print('Train Dataset: {}'.format(len(train_dataset)))
print('Val Dataset: {}'.format(len(val_dataset)))
print('Test Dataset: {}'.format(len(test_dataset)))

Cloning into 'Homework2-Caltech101'...
remote: Enumerating objects: 9256, done.
remote: Total 9256 (delta 0), reused 0 (delta 0), pack-reused 9256
Receiving objects: 100% (9256/9256), 129.48 MiB | 13.38 MiB/s, done.
Resolving deltas: 100% (4/4), done.
Checking out files: 100% (9149/9149), done.
Compose(
    Resize(size=256, interpolation=PIL.Image.BILINEAR)
    CenterCrop(size=(224, 224))
    ToTensor()
    Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
)
Train Dataset: 3886
Val Dataset: 1898
Test Dataset: 2893


**Prepare Dataloaders**

In [0]:
# Dataloaders iterate over pytorch datasets and transparently provide useful functions (e.g. parallelization and shuffling)
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, drop_last=True)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
test_dataloader = DataLoader(test_dataset, batch_size=8, shuffle=False, num_workers=4)

**Prepare Network**

In [0]:
#net = alexnet() # Loading AlexNet model
net = alexnet(pretrained=True) # Loading AlexNet model
#net = resnet50(pretrained=True)
# for param in net.parameters():
#      param.requires_grad = False
# AlexNet has 1000 output neurons, corresponding to the 1000 ImageNet's classes
# We need 101 outputs for Caltech-101
net.classifier[6] = nn.Linear(4096, NUM_CLASSES) # nn.Linear in pytorch is a fully connected layer
                                           # The convolutional layer is nn.Conv2d
# num_ftrs = net.fc.in_features
# net.fc = nn.Linear(num_ftrs, NUM_CLASSES)
# for param in net.classifier.parameters():
#      param.requires_grad = False

# We just changed the last layer of AlexNet with a new fully connected layer with 101 outputs
# It is mandatory to study torchvision.models.alexnet source code

Downloading: "https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth" to /root/.cache/torch/checkpoints/alexnet-owt-4df8aa71.pth
100%|██████████| 233M/233M [00:01<00:00, 171MB/s]


**Prepare Training**

In [0]:
# Define loss function
criterion = nn.CrossEntropyLoss() # for classification, we use Cross Entropy

# Choose parameters to optimize
# To access a different set of parameters, you have to access submodules of AlexNet
# (nn.Module objects, like AlexNet, implement the Composite Pattern)
# e.g.: parameters of the fully connected layers: net.classifier.parameters()
# e.g.: parameters of the convolutional layers: look at alexnet's source code ;) 
parameters_to_optimize = net.parameters() # In this case we optimize over all the parameters of AlexNet

# Define optimizer
# An optimizer updates the weights based on loss
# We use SGD with momentum
optimizer = optim.SGD(parameters_to_optimize, lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)

# Define scheduler
# A scheduler dynamically changes learning rate
# The most common schedule is the step(-down), which multiplies learning rate by gamma every STEP_SIZE epochs
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE, gamma=GAMMA)

**Train**

In [0]:
# By default, everything is loaded to cpu
net = net.to(DEVICE) # this will bring the network to GPU if DEVICE is cuda

cudnn.benchmark # Calling this optimizes runtime
best_model= net
max_acc = 0.0
bepoch = 0
current_step = 0
# Start iterating over the epochs
for epoch in range(NUM_EPOCHS):
  print('Starting epoch {}/{}, LR = {}'.format(epoch+1, NUM_EPOCHS, scheduler.get_lr()))
  net.train(True)
  # Iterate over the dataset
  for images, labels in train_dataloader:
    # Bring data over the device of choice
    images = images.to(DEVICE)
    labels = labels.to(DEVICE)

    net.train() # Sets module in training mode

    # PyTorch, by default, accumulates gradients after each backward pass
    # We need to manually set the gradients to zero before starting a new iteration
    optimizer.zero_grad() # Zero-ing the gradients

    # Forward pass to the network
    outputs = net(images)

    # Compute loss based on output and ground truth
    loss = criterion(outputs, labels)

    # Log loss
    if current_step % LOG_FREQUENCY == 0:
      print('Step {}, Loss {}'.format(current_step, loss.item()))

    # Compute gradients for each layer and update weights
    loss.backward()  # backward pass: computes gradients
    optimizer.step() # update weights based on accumulated gradients

    current_step += 1
  net.train(False)
  running_corrects = 0
  for images, labels in tqdm(val_dataloader):
    images = images.to(DEVICE)
    labels = labels.to(DEVICE)

    # Forward Pass
    outputs = net(images)
    #Get predictions
    _, preds = torch.max(outputs.data, 1)
  # for images, labels in tqdm(val_dataloader):
  #   images = images.to(DEVICE)
  #   labels = labels.to(DEVICE)

  #   bs, ncrops, c, h, w = images.size()
  #   result = net(images.view(-1, c, h, w)) # fuse batch size and ncrops
  #   result_avg = result.view(bs, ncrops, -1).mean(1) # avg over crops
  #   _, preds = torch.max(result_avg, 1)

  #   # Update Corrects
    running_corrects += torch.sum(preds == labels.data).data.item()

  # Calculate Accuracy
  accuracy = running_corrects / float(len(val_dataset))
  if(accuracy >= max_acc):
    max_acc = accuracy
    best_model = net
    bepoch = epoch+1
  print('\nEpoch:'+str(epoch+1) +' Validation Accuracy: {}'.format(accuracy))

  # Step the scheduler
  scheduler.step() 

Starting epoch 1/20, LR = [0.01]
Step 0, Loss 4.918406009674072
Step 10, Loss 1.9492219686508179


100%|██████████| 8/8 [00:07<00:00,  1.01s/it]


Epoch:1 Validation Accuracy: 0.7181243414120126
Starting epoch 2/20, LR = [0.01]


Step 20, Loss 0.5988706946372986


100%|██████████| 8/8 [00:07<00:00,  1.01it/s]

...

100%|██████████| 8/8 [00:07<00:00,  1.01it/s]


Epoch:19 Validation Accuracy: 0.8640674394099052
Starting epoch 20/20, LR = [0.001]


Step 290, Loss 0.0025726333260536194


100%|██████████| 8/8 [00:07<00:00,  1.01it/s]


Epoch:20 Validation Accuracy: 0.8661749209694415


**Test**

In [0]:
print("Using model with n_epochs: "+str(bepoch)+" with accuracy "+str(max_acc))
best_model = best_model.to(DEVICE) # this will bring the network to GPU if DEVICE is cuda
best_model.train(False) # Set Network to evaluation mode

running_corrects = 0
# for images, labels in tqdm(test_dataloader):
#   images = images.to(DEVICE)
#   labels = labels.to(DEVICE)

#   # Forward Pass
#   outputs = net(images)

#   # Get predictions
#   _, preds = torch.max(outputs.data, 1)

  # Update Corrects
  #   running_corrects += torch.sum(preds == labels.data).data.item()
for images, labels in tqdm(test_dataloader):
  images = images.to(DEVICE)
  labels = labels.to(DEVICE)

  bs, ncrops, c, h, w = images.size()
  result = best_model(images.view(-1, c, h, w)) # fuse batch size and ncrops
  result_avg = result.view(bs, ncrops, -1).mean(1) # avg over crops
  _, preds = torch.max(result_avg, 1)


  # Update Corrects
  running_corrects += torch.sum(preds == labels.data).data.item()

#   # Calculate Accuracy
# Calculate Accuracy
accuracy = running_corrects / float(len(test_dataset))

print('Test Accuracy: {}'.format(accuracy))

  0%|          | 0/362 [00:00<?, ?it/s]

Using model with n_epochs: 17 with accuracy 0.8672286617492097


100%|██████████| 362/362 [00:54<00:00,  6.63it/s]

Test Accuracy: 0.8631178707224335
